# Installation and downloads to work with scBasset
**Authorship:**
Adam Klie (last updated: *09/21/2023*)
***
**Description:**
Set-up computational environments and data for working with scBasset models from multiple sources.
- scBasset 
- EUGENe
- Kipoi
***

# Setting up your computational environment

```bash
# Create base conda 
conda create -n scbasset python=3.7
conda activate scbasset

# Install via pip in ML4GLand
cd /cellar/users/aklie/opt/ml4gland
git clone https://github.com/calico/scBasset.git
pip install -e scBasset

pip install protobuf==3.20
PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

# Trying hard to get a GPU working
conda install -c anaconda cudatoolkit=10.2
conda install -c anaconda cudatoolkit=11.3
conda install -c conda-forge cudnn==8.2.1
conda uninstall cudatoolkit cudnn -y
conda install -c anaconda cudatoolkit=11.2 cudnn=8.1 -y
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/scbasset/lib
# SEEMS TO WORK

# Get a kernel going!
python -m ipykernel install --user --name scbasset --display-name "Python 3.7 scbasset"
```

# Downloading scBasset tutorial data

In [ ]:
import os
import subprocess

download_savepath = '/cellar/users/aklie/data/ml4gland/use_cases/yuan22/github_tutorial'
os.makedirs(download_savepath, exist_ok=True)

if not os.path.exists('%s/buen_ad_sc.h5ad'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/buen_ad_sc.h5ad'%download_savepath, shell=True)

if not os.path.exists('%s/buen_model_sc.h5'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/buen_model_sc.h5'%download_savepath, shell=True)

if not os.path.exists('%s/pbmc_ad.h5ad'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_ad.h5ad'%download_savepath, shell=True)

if not os.path.exists('%s/pbmc_model.h5'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_model.h5'%download_savepath, shell=True)

# download motif injection fasta files for CISBP-1.0 motifs and unzip:
if not os.path.exists('%s/Homo_sapiens_motif_fasta.tar.gz'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/Homo_sapiens_motif_fasta.tar.gz'%download_savepath, shell=True)
subprocess.run('tar -xzf %s/Homo_sapiens_motif_fasta.tar.gz -C %s/'%(download_savepath, download_savepath), shell=True)s